In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from transformers import AlbertTokenizerFast, AlbertModel
# from gensim.models import Word2Vec
# from nets import *

In [2]:
# https://zh-v2.d2l.ai/chapter_natural-language-processing-applications/sentiment-analysis-cnn.html
class GloveEmbedding:
    """Token Embedding."""

    def __init__(self):
        self.idx_to_token, self.idx_to_vec = self._load_embedding()
        self.unknown_idx = 0
        # a dictionary like {'best':1} from idx_to_token
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = '../../Datasets/Model/nlp/'
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), encoding='gb18030', errors='ignore') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                # Skip header information, such as the top row in fastText
                # structure: hello  [0.1, 0.2, 0.3, ...]
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [
            self.token_to_idx.get(token, self.unknown_idx)
            for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [2]:
'''CONSTANTS'''
batch_size = 32
max_epoch = 40
lr = 0.001

LD = 30
LW = 20

device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
train_data_path = '../../Datasets/Model/train'
test_data_path = '../../Datasets/Model/test'
image_path = '../../Datasets/Images'
data_save_path = '../../Save/Experiment5'
analytics_path = '../../Analysis/Experiment5'
nlp_path = '../../Datasets/Model/nlp'

'''NLP'''
# load word2vec models from nlp_path
# model_review = Word2Vec.load(f'{nlp_path}/model_review.model')
# model_title = Word2Vec.load(f'{nlp_path}/model_title.model')

# get the vocabulary of reviews titles
# vocab_review = model_review.wv.index_to_key
# vocab_title = model_title.wv.index_to_key

# # get the vectors of reviews and titles
# vecs_review = torch.tensor(model_review.wv[vocab_review])
# vecs_title = torch.tensor(model_title.wv[vocab_title])

# get the glove embeddings
# print("Loading glove embeddings...")
# glove = GloveEmbedding()


'''ANALYTICS'''

rmse_arr = []


In [3]:
device

device(type='cuda')

In [3]:
train_users = np.load(f'{train_data_path}/train_users.npy')
train_items = np.load(f'{train_data_path}/train_items.npy')
train_ratings = np.load(f'{train_data_path}/train_ratings.npy')
train_reviews = np.load(
    f'{train_data_path}/train_reviews.npy', allow_pickle=True)
train_descriptions = np.load(
    f'{train_data_path}/train_descriptions.npy', allow_pickle=True)
train_titles = np.load(
    f'{train_data_path}/train_titles.npy', allow_pickle=True)
train_prices = np.load(f'{train_data_path}/train_prices.npy')
train_categories = np.load(f'{train_data_path}/train_categories.npy')

test_users = np.load(f'{test_data_path}/test_users.npy')
test_items = np.load(f'{test_data_path}/test_items.npy')
test_ratings = np.load(f'{test_data_path}/test_ratings.npy')
test_reviews = np.load(f'{test_data_path}/test_reviews.npy', allow_pickle=True)
test_descriptions = np.load(
    f'{test_data_path}/test_descriptions.npy', allow_pickle=True)
test_titles = np.load(
    f'{test_data_path}/test_titles.npy', allow_pickle=True)
test_prices = np.load(f'{test_data_path}/test_prices.npy')
test_categories = np.load(f'{test_data_path}/test_categories.npy')


In [4]:
max(test_items)

72693

In [4]:
# this function accepts a sequence, and pads the setences in the sequence to LW length, then returns the padded sequence
def pad_sequence(sequence, lw = LW, ld = LD, placeholder=0):
    arr = []
    for sentence in sequence:
      if len(sentence) < lw:
        arr.append(sentence + [placeholder] * (lw - len(sentence)))
      else:
        arr.append(sentence[:lw])

    if len(arr) > ld:
      arr = arr[:ld]
    else:
      for i in range (ld-len(arr)):
        arr.append([placeholder] * lw)
        
    return arr

'''sample user'''
'''this function takes a user, who is from a set of users, a set of items, a set of reviews, a set of titles, 
a set of categories, a set of prices, and a set of ratings.'''
def sample_user(user, users, items, reviews, ratings):
  # find out all the indicies of the user in users
  user_indices = np.where(users == user)[0]
  # randomly select an index from these indicies.
  user_index = np.random.choice(user_indices)

  # get the item that the user has purchased.
  item = items[user_index]

  # get the rating that the user has given to that item.
  rating = ratings[user_index]

  # find out the indices of the item in items
  item_indices = np.where(items == item)[0]

  # get the reviews of that item
  item_reviews = [reviews[i] for i in item_indices if i != user_index]
  user_reviews = [reviews[i] for i in user_indices if i != user_index]

  user_reviews = pad_sequence(user_reviews)
  item_reviews = pad_sequence(item_reviews)

  return user, item, rating, item_reviews, user_reviews


'''function to get a batch of training samples.
this function selects a random user from a set of users
and gets his information by function sample_user, repeat it for batch_size times and 
gets the batch of samples. Each element in the batch tuple is then transormed to a pytorch tensor and returned.'''
def get_batch(users, items, reviews, ratings, batch_size=32, fixed_users_set = None):
  # get a batch of users
  if fixed_users_set is None:
    batch_users = np.random.choice(users, size=batch_size)
  else:
    batch_users = fixed_users_set
  # get the batch of samples
  batch = [list(sample_user(user, users, items, reviews, ratings)) for user in batch_users]
  # transform each column of the batch to a pytorch tensor
  batch = [torch.tensor(sample) for sample in zip(*batch)]
  return batch



In [5]:
# get a batch of training samples
batch = get_batch(train_users, train_items, train_reviews, train_ratings)
user, item, rating, item_reviews, user_reviews = batch

In [23]:
def weights_init(m):
  if isinstance(m, nn.Linear):
    nn.init.normal_(m.weight.data)
    nn.init.constant_(m.bias.data, 0.0)

class FM(nn.Module):
    def __init__(self, latent_dim, fea_num):
        super().__init__()

        self.latent_dim = latent_dim
        self.w0 = nn.Parameter(torch.zeros([1, ], requires_grad=True))
        self.w1 = nn.Parameter(torch.rand([fea_num, 1], requires_grad=True))
        self.w2 = nn.Parameter(torch.rand([fea_num, latent_dim], requires_grad=True))

    def forward(self, inputs):
        # inputs = inputs.long()
        first_order = self.w0 + torch.mm(inputs, self.w1)
        second_order = 1/2 * torch.sum(
            torch.pow(torch.mm(inputs, self.w2), 2) -
            torch.mm(torch.pow(inputs, 2), torch.pow(self.w2, 2)),

            dim=1,
            keepdim=True
        )

        return first_order + second_order


class DeepFM(nn.Module):
  def __init__(self, fea_num, latent_dim):

    super().__init__()
    
    self.w0 = nn.Parameter(torch.zeros([1, ]))
    self.w1 = nn.Parameter(torch.rand([fea_num, 1]))
    self.w2 = nn.Parameter(torch.rand([fea_num, latent_dim]))

    self.mlp = nn.Sequential(
        nn.Linear(latent_dim, 64),
        nn.ReLU(inplace=True),
        nn.Linear(64, 32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 16),
        nn.ReLU(inplace=True),
        nn.Linear(16, latent_dim),
        nn.ReLU(inplace=True),
        nn.Linear(latent_dim, 1)
    )

  def forward(self, inputs):
    '''FM'''
    fm_first_order = self.w0 + torch.mm(inputs, self.w1)
    fm_second_order = 1/2 * torch.sum(
        torch.pow(torch.mm(inputs, self.w2), 2)
        -
        torch.mm(torch.pow(inputs, 2), torch.pow(self.w2, 2)),
        dim=1,
        keepdim=True
    )
    fm_out = fm_first_order + fm_second_order

    '''MLP'''
    mlp_x = torch.mm(inputs, self.w2)
    mlp_out = self.mlp(mlp_x)

    return torch.add(fm_out, mlp_out) / 2
      


class MPCN(nn.Module):
  def __init__(self, vocab_size = 100000, embed_size = 50, lw = LW, fm_dim = 8, np=1):
      super().__init__()
      self.np = np
      self.embedding = nn.Embedding(vocab_size, embed_size)
      # initialise a weighting matrix of size (50 x 50)
      self.W_g = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      # initialise a bias vector of size (50)
      self.b_g = nn.Parameter(torch.zeros(embed_size, requires_grad=True))
      # initialise a weighting matrix of size (50 x 50)
      self.W_u = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      # initialise a bias vector of size (50)
      self.b_u = nn.Parameter(torch.zeros(embed_size, requires_grad=True))
      self.sigmoid = nn.Sigmoid()
      self.tanh = nn.Tanh()

      self.F_review = nn.Sequential(
        nn.Linear(embed_size, 2*embed_size, bias=True),
        nn.Sigmoid(),
        nn.Dropout(0.2),
        nn.Linear(2*embed_size, embed_size, bias=True),
        nn.Sigmoid(),
        nn.Dropout(0.2),
      )

      self.F_word = nn.Sequential(
        nn.Linear(embed_size, 2*embed_size, bias=True),
        nn.Sigmoid(),
        nn.Dropout(0.2),
        nn.Linear(2*embed_size, embed_size, bias=True),
        nn.Sigmoid(),
        nn.Dropout(0.2),
      )

      # initialise a weighting matrix of size (50 x 50)
      self.M = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      self.M_w = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)

      self.deepfm = DeepFM(2*embed_size, fm_dim)

  def forward(self, a, b):
    a = self.embedding(a.long())
    b = self.embedding(b.long())
    # size of a: (32, 30, 20, 50)
    # size of b: (32, 30, 20, 50)

    a_sumed = torch.sum(a, dim=2)
    b_sumed = torch.sum(b, dim=2)
    # size of a_sumed: (32, 30, 50)
    # size of b_sumed: (32, 30, 50)

    a_bar = self.sigmoid(a_sumed @ self.W_u) + self.b_g * self.tanh(a_sumed @ self.W_g + self.b_u)
    b_bar = self.sigmoid(b_sumed @ self.W_u) + self.b_g * self.tanh(b_sumed @ self.W_g + self.b_u)
    # size of a_bar: (32, 30, 50)
    # size of b_bar: (32, 30, 50)

    a_prime_arr = []
    b_prime_arr = []


    for i in range (self.np):
      S = self.F_review(a_bar) @ self.M @ self.F_review(b_bar).permute(0,2,1)
      # get the maximum of S columnwise
      max_col_S = torch.max(S, dim=1)[0]
      # get the maximum of S rowwise
      max_row_S = torch.max(S, dim=2)[0]

      # size of S: (32, 30, 30)
      # size of max_col_S: (32, 30)
      # size of max_row_S: (32, 30)

      p_a = nn.functional.gumbel_softmax(logits=max_col_S, hard=True, dim=-1, tau=0.1).unsqueeze(1).unsqueeze(2)
      p_b = nn.functional.gumbel_softmax(logits=max_row_S, hard=True, dim=-1, tau=0.1).unsqueeze(1).unsqueeze(2)
      # size of p_a: (32, 1, 1, 30)
      # size of p_b: (32, 1, 1, 30)

      ''' This step implements the pointer selection mechanism in the paper 
      
      p_a is originally a two-dimensional tensor of size (32 x 30(LW)), while a, b \
      are originally a four-dimensional tensor of size (32 x 30(LW) x 20 x 50) \
      we need to reshape p_a and p_b to a four-dimensional tensor of size (32, 1, 1, 30) \
      and multiply with reshaped a and b to (32, 20, 30, 50) (a.permute(0,2,1,3)) \
      then we get the correct but not ideally-shaped selected review of size a' = (32, 1, 20, 50) \
      then we squeeze the second dimension (a'.squeeze(1)) of a' to get (32, 20, 50)
      
      ''' 

      a_prime = (p_a @ a.permute(0,2,1,3)).permute(0,2,1,3).squeeze(1)
      b_prime = (p_b @ b.permute(0,2,1,3)).permute(0,2,1,3).squeeze(1)
      # size of a_prime: (32, 20, 50)
      # size of b_prime: (32, 20, 50)

      a_prime_arr.append(a_prime)
      b_prime_arr.append(b_prime)
    
    a_prime = torch.cat(a_prime_arr, dim=1)
    b_prime = torch.cat(b_prime_arr, dim=1)
    
    W = self.F_word(a_prime) @ self.M_w @ self.F_word(b_prime).permute(0,2,1)
    # size of W: (32, 20, 20)

    # get the column average of W
    avg_col_W = torch.mean(W, dim=1).unsqueeze(2)
    # get the row average of W
    avg_row_W = torch.mean(W, dim=2).unsqueeze(2)
    # size of avg_col_W: (32, 20, 1)
    # size of avg_row_W: (32, 20, 1)
    
    a_bar_prime = self.sigmoid(avg_col_W).permute(0,2,1) @ a_prime
    b_bar_prime = self.sigmoid(avg_row_W).permute(0,2,1) @ a_prime
    # size of a_bar_prime: (32, 1, 50)
    # size of b_bar_prime: (32, 1, 50)

    a_bar_prime = a_bar_prime.squeeze(1)
    b_bar_prime = b_bar_prime.squeeze(1)

    # concatenate a_bar_prime and b_bar_prime
    a_bar_prime_bar_prime = torch.cat((a_bar_prime, b_bar_prime), dim=1)
    
    pred = self.deepfm(a_bar_prime_bar_prime)

    # flatten the predictions
    pred = pred.squeeze(1)

    return pred

In [24]:
mpcn = MPCN().apply(weights_init).to(device)
optimiser_mpcn = optim.Adam(mpcn.parameters(), lr=lr, weight_decay=1e-6)

def save_training(path = f'{data_save_path}/'):
  torch.save(mpcn.state_dict(), path + 'mpcn_p1.pt')
  torch.save(optimiser_mpcn.state_dict(), path + 'mpcn_optimiser_p1.pt')
  print('Saved model and optimiser')

def load_training(path = f'{data_save_path}/'):
  mpcn.load_state_dict(torch.load(path + 'mpcn_p1.pt'))
  optimiser_mpcn.load_state_dict(torch.load(path + 'mpcn_optimiser_p1.pt'))
  print('Loaded model and optimiser')

In [25]:
'''Training'''
RMSE = []
best_rmse = 1e6

def evaluation(test_batch_size = 32):
  test_users_unique = list(set(test_users))
  rmse_arr = []
  for i in tqdm(range(0, len(test_users_unique), test_batch_size)):
    test_users_batch = test_users_unique[i:i+test_batch_size]
    batch_test = get_batch(test_users, test_items, test_reviews, test_ratings, fixed_users_set=test_users_batch)
    user, item, rating, item_reviews, user_reviews = batch_test
    user, item, rating, item_reviews, user_reviews = user.to(device), item.to(device), rating.to(device), item_reviews.to(device), user_reviews.to(device)

    pred = mpcn(item_reviews, user_reviews)

    # calculate rooted mean square error
    rmse_arr.append(torch.sqrt(torch.mean((pred - rating)**2)).item())
  
  # return the mean of the rmse_arr
  return np.mean(rmse_arr)

print("Program: Evaluating the baseline RMSE of the model on the test set")

# set best_rmse to be the largest possible value
best_rmse = evaluation()
RMSE.append(best_rmse)

# print baseline best_rmse
print('Baseline RMSE:', best_rmse)

for epoch in range (max_epoch):
  for i in tqdm(range(1000)):
    batch =  get_batch(train_users, train_items, train_reviews, train_ratings)
    user, item, rating, item_reviews, user_reviews = batch
    user, item, rating, item_reviews, user_reviews = user.to(device), item.to(device), rating.to(device), item_reviews.to(device), user_reviews.to(device)

    pred = mpcn(item_reviews, user_reviews)

    # calculate the loss
    loss = torch.mean((pred - rating)**2)

    optimiser_mpcn.zero_grad()
    loss.backward()
    optimiser_mpcn.step()

  '''evaluate the model on the test set'''
  rmse_test = evaluation()
  RMSE.append(rmse_test)
  # print out the loss of the models and the test rmse
  print('Epoch:', epoch, 'Loss:', loss.item(), 'Test RMSE:', rmse_test)
  if rmse_test < best_rmse:
    best_rmse = rmse_test
    save_training()


Program: Evaluating the baseline RMSE of the model on the test set


100%|██████████| 3609/3609 [00:56<00:00, 64.44it/s]


Baseline RMSE: 6383.381097900864


100%|██████████| 3609/3609 [00:57<00:00, 62.90it/s]


Epoch: 0 Loss: 62.45726776123047 Test RMSE: 6.853791579608278
Saved model and optimiser


...

Epoch: 37 Loss: 1.4667139053344727 Test RMSE: 1.161170880313592


100%|██████████| 3609/3609 [00:47<00:00, 75.43it/s]


Epoch: 38 Loss: 1.0771491527557373 Test RMSE: 1.1592055894589286


100%|██████████| 3609/3609 [00:48<00:00, 75.13it/s]

Epoch: 39 Loss: 0.43612024188041687 Test RMSE: 1.1822737908389767


In [26]:
with open (f'{analytics_path}/rmse_p1_deepfm.txt', 'w') as f:
  for r in RMSE:
    f.write(str(np.round(r, 4)) + '\n')